## More questions:
> 1- Work challenges

In [201]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns
import plotly.express as px
%matplotlib inline

In [202]:
df = pd.read_csv('survey_results_public_2019.csv', encoding =  "ISO-8859-1", low_memory=False)

In [203]:
df = df[df['WorkRemote'].notna()]

In [204]:
df['WorkRemote'].value_counts()

Less than once per month / Never                           30220
A few days each month                                      17242
All or almost all the time (I'm full-time remote)           8465
Less than half the time, but at least one day each week     6320
It's complicated                                            3675
More than half, but not all, the time                       2376
About half the time                                         1986
Name: WorkRemote, dtype: int64

In [205]:
df['WorkRemote'].replace("All or almost all the time (I'm full-time remote)", "Full-time remote", inplace = True)
df['WorkRemote'].replace("More than half, but not all, the time", "Part-time remote", inplace = True)
df['WorkRemote'].replace("About half the time", "Part-time remote", inplace = True)
df['WorkRemote'].replace("Less than half the time, but at least one day each week", "Part-time remote", inplace = True)
df['WorkRemote'].replace("Less than once per month / Never", "Rarely or never", inplace = True)

#### 1- Work Challenges

In [206]:
df_challenge = df[df['WorkChallenge'].notna()]

In [207]:
from collections import defaultdict
def total_count(df, col1, col2, look_for):
    '''
    INPUT:
    df - the pandas dataframe you want to search
    col1 - the column name you want to look through
    col2 - the column you want to count values from
    look_for - a list of strings you want to search for in each row of df[col]
    
    OUTPUT:
    new_df - a dataframe of each look_for with the count of how often it shows up 
    '''
    new_df = defaultdict(int)
    for val in look_for:
        for idx in range(df.shape[0]):
            if val in df[col1][idx]:
                new_df[val] += int(df[col2][idx])   
    new_df = pd.DataFrame(pd.Series(new_df)).reset_index()
    new_df.columns = [col1, col2]
    new_df.sort_values('count', ascending=False, inplace=True)
    return new_df

df_never = df_challenge[df_challenge['WorkRemote']=='Rarely or never']['WorkChallenge'].value_counts().reset_index()
df_never.rename(columns={'index':'challenge', 'WorkChallenge':'count'}, inplace=True)
possible_values = set()
df_never['challenge'].apply(lambda x : [possible_values.add(item) for item in x.split(';')])
df_never = total_count(df_never, 'challenge', 'count', possible_values).copy()
    
    
df_afew = df_challenge[df_challenge['WorkRemote']=='A few days each month']['WorkChallenge'].value_counts().reset_index()
df_afew.rename(columns={'index':'challenge', 'WorkChallenge':'count'}, inplace=True)
possible_values = set()
df_afew['challenge'].apply(lambda x : [possible_values.add(item) for item in x.split(';')])
df_afew = total_count(df_afew, 'challenge', 'count', possible_values).copy()

df_part = df_challenge[df_challenge['WorkRemote']=='Part-time remote']['WorkChallenge'].value_counts().reset_index()
df_part.rename(columns={'index':'challenge', 'WorkChallenge':'count'}, inplace=True)
possible_values = set()
df_part['challenge'].apply(lambda x : [possible_values.add(item) for item in x.split(';')])
df_part = total_count(df_part, 'challenge', 'count', possible_values).copy()

df_full = df_challenge[df_challenge['WorkRemote']=='Full-time remote']['WorkChallenge'].value_counts().reset_index()
df_full.rename(columns={'index':'challenge', 'WorkChallenge':'count'}, inplace=True)
possible_values = set()
df_full['challenge'].apply(lambda x : [possible_values.add(item) for item in x.split(';')])
df_full = total_count(df_full, 'challenge', 'count', possible_values).copy()

df_comp = df_challenge[df_challenge['WorkRemote']=="It's complicated"]['WorkChallenge'].value_counts().reset_index()
df_comp.rename(columns={'index':'challenge', 'WorkChallenge':'count'}, inplace=True)
possible_values = set()
df_comp['challenge'].apply(lambda x : [possible_values.add(item) for item in x.split(';')])
df_comp = total_count(df_comp, 'challenge', 'count', possible_values).copy()

In [208]:
from functools import reduce
dfs = [df_never, df_afew, df_part, df_full, df_comp]
df_final = reduce(lambda left,right: pd.merge(left,right,on='challenge'), dfs)
df_final.columns = ['challenge', 'Never', 'A few times a month', 'Part-time', 'Full-time', 'Complicated']
df_final['Never'] = df_final['Never'] / df_final['Never'].sum()
df_final['A few times a month'] = df_final['A few times a month'] / df_final['A few times a month'].sum()
df_final['Part-time'] = df_final['Part-time'] / df_final['Part-time'].sum()
df_final['Full-time'] = df_final['Full-time'] / df_final['Full-time'].sum()
df_final['Complicated'] = df_final['Complicated'] / df_final['Complicated'].sum()

In [209]:
df_final_melt = df_final.melt(value_vars=['Never','A few times a month','Part-time','Full-time','Complicated'], id_vars=['challenge'])
df_final_melt.head()

,challenge,variable,value
0,Distracting work environment,Never,0.165322
1,Being tasked with non-development work,Never,0.147507
2,Not enough people for the workload,Never,0.131567
3,Meetings,Never,0.130777
4,Lack of support from management,Never,0.111062


In [210]:

fig = px.bar(df_final_melt, x='variable', y='value',barmode="group", color = 'challenge')
fig.update_layout(
    title="Challenges selected by each group",
    yaxis_title="Percentage of selected challenge",
    xaxis_title="Remote work options",
    autosize=False,
    width=1500,
    height=600,)
fig.show()
fig.write_image("Challenges.png")

#### 2- Operating system

In [211]:
df_opSys = df[df['OpSys'].notna()]

In [212]:
df_os = df_opSys.groupby(['WorkRemote','OpSys']).count()['Respondent'].reset_index()
df_os.rename(columns={'Respondent':'count'}, inplace = True)
df_os_per = df_os.groupby(['WorkRemote','OpSys']).agg({'count': 'sum'})
df_os_per = df_os_per.groupby(level=0).apply(lambda x:100 * x / float(x.sum())).reset_index()
df_os_per = df_os_per[df_os_per['OpSys']!='BSD']
df_os_per.head()

,WorkRemote,OpSys,count
1,A few days each month,Linux-based,24.221211
2,A few days each month,MacOS,34.651733
3,A few days each month,Windows,41.045386
5,Full-time remote,Linux-based,27.140645
6,Full-time remote,MacOS,37.608670


In [213]:
fig = px.bar(df_os_per, x='WorkRemote', y='count',barmode="group", color = 'OpSys')
fig.update_layout(
    yaxis_title="Percentage of OS used",
    xaxis_title="Remote work options",
    autosize=False,
    width=800,
    height=400,)
fig.show()
fig.write_image("os.png")

#### 3- Country

In [214]:
df_coun = df[df['Country'].notna()]

In [215]:
df_coun = df_coun[df.Country.isin(['United States','India','Germany','United Kingdom','Canada'])]

In [216]:
df_country = df_coun.groupby(['WorkRemote','Country']).count()['Respondent'].reset_index()
df_country.rename(columns={'Respondent':'count'}, inplace = True)
df_country_per = df_country.groupby(['Country','WorkRemote']).agg({'count': 'sum'})
df_country_per = df_country_per.groupby(level=0).apply(lambda x:100 * x / float(x.sum())).reset_index()

In [217]:
df_country_per[df_country_per['WorkRemote'] == 'Full-time remote'].sort_values('count', ascending = False)

,Country,WorkRemote,count
21,United States,Full-time remote,15.846215
1,Canada,Full-time remote,13.526570
16,United Kingdom,Full-time remote,9.004739
6,Germany,Full-time remote,6.933220
11,India,Full-time remote,6.729598


In [218]:
fig = px.bar(df_country_per, x='Country', y='count',barmode="group", color = 'WorkRemote')
fig.update_layout(
    yaxis_title="Percentage",
    xaxis_title="Country",
    autosize=False,
    width=800,
    height=400,)
fig.show()
fig.write_image("country.png")

#### Company size

In [219]:
df_OrgSize = df[df['OrgSize'].notna()]


In [220]:
df_Org = df_OrgSize.groupby(['WorkRemote','OrgSize']).count()['Respondent'].reset_index()
df_Org.rename(columns={'Respondent':'count'}, inplace = True)
df_Org_per = df_Org.groupby(['OrgSize','WorkRemote']).agg({'count': 'sum'})
df_Org_per = df_Org_per.groupby(level=0).apply(lambda x:100 * x / float(x.sum())).reset_index()

In [221]:
df_Org_per.head()

,OrgSize,WorkRemote,count
0,"1,000 to 4,999 employees",A few days each month,27.406380
1,"1,000 to 4,999 employees",Full-time remote,5.783634
2,"1,000 to 4,999 employees",It's complicated,4.466019
3,"1,000 to 4,999 employees",Part-time remote,14.965326
4,"1,000 to 4,999 employees",Rarely or never,47.378641


In [222]:
fig = px.bar(df_Org_per, x='OrgSize', y='count',barmode="group", color = 'WorkRemote')
fig.update_layout(
    yaxis_title="Percentage",
    xaxis_title="Number of employees",
    autosize=False,
    width=1000,
    height=600,)
fig.show()
fig.write_image("org.png")

#### Social media

In [229]:
df_social_media['SocialMedia'].value_counts()

Reddit                      11814
Facebook                    10749
WhatsApp                    10641
Twitter                      9915
YouTube                      9737
Instagram                    4921
I don't use social media     4478
LinkedIn                     3745
WeChat å¾®ä¿¡                 494
VK ÐÐÐ¾Ð½ÑÐ°ÌÐºÑÐµ       451
Snapchat                      350
Weibo æ°æµªå¾®å             39
Youku Tudou ä¼é
·             17
Hello                           9
Name: SocialMedia, dtype: int64

In [237]:
df_social_media = df[df['SocialMedia'].notna()]
df_social_media = df_social_media[df_social_media.SocialMedia.isin(['Reddit','Facebook','WhatsApp','Twitter','YouTube'])]

In [238]:
df_social = df_social_media.groupby(['WorkRemote','SocialMedia']).count()['Respondent'].reset_index()
df_social.rename(columns={'Respondent':'count'}, inplace = True)
df_social_per = df_social.groupby(['WorkRemote','SocialMedia']).agg({'count': 'sum'})
df_social_per = df_social_per.groupby(level=0).apply(lambda x:100 * x / float(x.sum())).reset_index()
df_social_per

,WorkRemote,SocialMedia,count
0,A few days each month,Facebook,20.183838
1,A few days each month,Reddit,24.458062
2,A few days each month,Twitter,19.088472
3,A few days each month,WhatsApp,19.502106
4,A few days each month,YouTube,16.767522
5,Full-time remote,Facebook,20.301587
6,Full-time remote,Reddit,19.825397
7,Full-time remote,Twitter,26.238095
8,Full-time remote,WhatsApp,15.666667
9,Full-time remote,YouTube,17.968254


In [240]:
fig = px.bar(df_social_per, x='WorkRemote', y='count',barmode="group", color = 'SocialMedia')
fig.update_layout(
    yaxis_title="Percentage",
    xaxis_title="Number of employees",
    autosize=False,
    width=1000,
    height=600,)
fig.show()
fig.write_image("socialMedia.png")